In [22]:
import torch
from torch.utils.data import DataLoader, Dataset
import os
from PIL import Image
import numpy as np
import utils
import matplotlib.pyplot as plt

In [23]:
args = {}
args["data_dir"] = "data/segmentation02/segmentation"
args["data"] = "segmentation02"

In [24]:
dataloaders, dataset_sizes = utils.get_dataloaders(
            args["data"], args["data_dir"], ["train", "val", "test"], batch_size=4, num_workers=8
        )

In [25]:
def calculate_bounding_box(mask):
    non_zero_indices = np.nonzero(mask)
    
    x_coords = non_zero_indices[1]
    y_coords = non_zero_indices[0]
    
    min_x = np.min(x_coords)
    max_x = np.max(x_coords)
    min_y = np.min(y_coords)
    max_y = np.max(y_coords)
    return (min_x, min_y, max_x, max_y)

In [28]:
files = ["list_train.txt", "list_val.txt", "list_test.txt"]
op_files = ["list_train_loc.txt", "list_val_loc.txt", "list_test_loc.txt"]
for ip_file, op_file in zip(files, op_files):
    with open(os.path.join(args["data_dir"], op_file), "w") as op_file:
        with open(os.path.join(args["data_dir"], ip_file), "r") as file:
            for line in file:
                line = line.strip()
                if line and line.split(",")[0] != "filenames":
                    file_path, label_path = line.split(",")
                    img_name = file_path.strip()
                    label_name = label_path.strip()
                    path = os.path.join(args["data_dir"], label_name)
                    mask = Image.open(path)
                    mask_arr = np.array(mask)
                    bb_heart = calculate_bounding_box((mask_arr == 85).astype("int"))
                    bb_lungs = calculate_bounding_box((mask_arr == 255).astype("int"))
                    op_file.write(file_path + ",heart," + ",".join(map(str, bb_heart)) + ",lungs," + ",".join(map(str, bb_lungs)) + "\n" )